## Loan Underwriter Multi Agent with CrewAI

Loan underwriting is a critical process that evaluates a borrower's financial stability and creditworthiness to determine loan approval. The process begins with the submission of key documents, including the loan application, which details the borrower's personal and financial information. Additional required documents typically include W-2 forms, credit reports, payslips, bank statements, and tax returns, all of which help underwriters assess the borrower’s ability to repay the loan. These documents provide insights into the borrower's employment history, income stability, outstanding debts, and overall financial obligations. A thorough review of financial information, including banking transactions, existing loans, payment history, and income sources, ensures that the borrower meets the lender’s risk assessment criteria.

Once all documentation is verified, the underwriting process follows a structured set of underwriting guidelines to determine the risk level associated with approving the loan. These guidelines typically include an evaluation of the Loan-to-Value (LTV) ratio, Debt-to-Income (DTI) ratio, credit score, and adherence to Fannie Mae and Freddie Mac lending standards. Underwriters also conduct fraud checks and verify compliance with Anti-Money Laundering (AML) regulations. The underwriting process may involve automated underwriting systems (AUS) to analyze the applicant's financial health and risk profile. After completing the assessment, the underwriter issues a decision—approval, conditional approval, or denial—based on the applicant’s financial capacity and adherence to lending requirements.

## CrewAI Implementation

By leveraging CrewAI, an agentic AI system can automate and streamline loan underwriting process using specialized AI agents. These agents can work collaboratively to extract, retrieve, and process financial information efficiently, reducing manual effort and improving accuracy. This system consists of four key agents: an OCR Agent for document processing, an SQL Agent for financial data retrieval, an Underwriter Guideline Agent for policy checks, and an Underwriter Supervisor Agent to coordinate the entire underwriting decision-making process.

<img src="crewai_underwriting_agent.png" alt="Loan Process" width="900" height="600"/>


**1. OCR Agent – Extracting Data from Loan Documents using Amazon Textract**

The OCR Agent is responsible for extracting and structuring data from key financial documents, such as loan applications, W-2 forms, credit reports, payslips, and tax returns. This agent integrates Amazon Textract, a powerful OCR tool, to scan and convert document-based information into structured digital formats. Once extracted, the data is cleaned and formatted for further processing by downstream agents.

**2. SQL Agent – Retrieving Loan Applicant’s Financial Data using Text2SQL**

The SQL Agent connects to a structured database (SQL-based system) to fetch relevant financial details of the loan applicant. Using Text2SQL conversion, this agent translates natural language queries (e.g., “What is the applicant’s monthly income and outstanding loan balance?”) into SQL queries that retrieve the necessary financial records. It extracts banking transactions, outstanding loan amounts, payment history, income sources, and credit utilization, providing a comprehensive financial profile of the borrower.

**3. Underwriter Guideline Agent – Ensuring Compliance with Loan Underwriting Policies**

The Underwriter Guideline Agent connects to Amazon Bedrock’s Knowledge Base to retrieve underwriting guidelines dynamically. By leveraging Retrieval-Augmented Generation (RAG), this agent provides contextual information regarding Loan-to-Value (LTV) ratios, Debt-to-Income (DTI) limits, credit score requirements, employment history criteria, and anti-money laundering (AML) checks. 

**4. Underwriter Supervisor Agent – Coordinating Agents and Generating Loan Decisions**

The Underwriter Supervisor Agent acts as the orchestrator, coordinating all the other agents to produce a final underwriting decision.It generates an underwriting report summarizing key factors, risk assessments, and policy adherence, ensuring transparency and explainability in loan decisions.

#### Install and Imports

In [1]:
!pip install -qr requirements.txt

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

#### TextractOCRTool - Amazon Textract OCR Extraction Tool

The `TextractOCRTool` is a custom tool that leverages **Amazon Textract** to extract text from all documents in a given folder. This tool is useful for processing loan applications, W-2 forms, credit reports, payslips, and other financial documents in an automated loan underwriting system. Checks the documents includes in `docs` folder

In [3]:
from crewai.tools import BaseTool
import boto3
import os

def extract_text_from_folder(folder_path: str) -> str:
    """
    Extracts and aggregates text from all documents in the specified folder using Amazon Textract.
    
    :param folder_path: Path to the folder containing documents.
    :return: Aggregated text from all documents.
    """
    textract_client = boto3.client('textract')
    aggregated_text = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
            with open(file_path, 'rb') as document:
                image_bytes = document.read()
            
            response = textract_client.detect_document_text(Document={'Bytes': image_bytes})
            
            lines = [block['Text'] for block in response['Blocks'] if block['BlockType'] == 'LINE']
            document_text = "\n".join(lines)
            
            aggregated_text.append(f"--- Text from {filename} ---\n{document_text}\n")
    
    return "\n".join(aggregated_text)

class TextractOCRTool(BaseTool):
    name: str = "textract_ocr_tool"
    description: str = "Custom tool that uses Amazon Textract to extract text from an image file."

    def _run(self, file_path: str) -> str:
        """
        Runs the OCR extraction on the provided file path.
        """
        try:
            text = extract_text_from_folder(file_path)
            return text
        except Exception as e:
            return f"Error during OCR extraction: {e}"

    def _arun(self, file_path: str) -> str:
        raise NotImplementedError("Asynchronous run not supported.")

#### OCR Agent and OCR Task for Loan Underwriting Automation

- The OCR Agent is designed to process loan-related documents such as W-2 forms, payslips, tax returns, and credit reports using Amazon Textract. It extracts and structures text from image-based financial documents into a predefined JSON format. This ensures that the underwriting process receives structured data without manual intervention.
- The OCR Task defines the structured output format expected from the OCR agent.


In [4]:
ocr_agent = Agent(
    role="OCR Agent",
    goal="""Extract text from image documents using Amazon Textract and structure the data as per the predefined JSON schema.""",
    backstory="An agent that leverages a custom OCR tool to process images and output the detected text in a structured JSON format.",
    tools=[TextractOCRTool()],
    verbose=True,
    llm=LLM(model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"),
)

ocr_task = Task(
    description="Extract and structure data from the document at path: {file_path}. Do not hallucinate and do not include information thats not in the documents",
    expected_output="""{
        "name": "string",
        "age": "integer",
        "SSN": "string",
        "phone_number": "string",
        "employment_information": {
            "employer": "string",
            "employer_address": "string",
            "position": "string",
            "years_employed": "integer"
        },
        "credit_information": {
            "credit_scores": "integer",
        },
        "property_information": {
            "current_residence": "string",
            "property_address": "string",
            "county": "string",
            "property_type": "string"
        },
        "loan_details": {
            "purchase_price": "float",
            "total_cost": "float",
            "loan_amount": "float"
        }
    }""",
    agent=ocr_agent
)

#### Text2SQLTool

The TextToSQLTool is a tool that enables natural language to SQL conversion for querying customer financial details from a SQLite database. It takes user queries in plain English, translates them into structured SQL statements, and executes them to retrieve relevant financial information. This tool is essential for automating loan underwriting, allowing underwriters to fetch details like bank credit score, late payments, foreclosures, bankrupcies, outstanding debts and payment history without manual SQL writing. By integrating with CrewAI agents, it ensures efficient and accurate financial data retrieval, streamlining the decision-making process in AI-driven loan evaluation.

In [5]:
import sqlite3
import re


def setup_database(db_path=":memory:"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE customer (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            credit_score INTEGER NOT NULL,
            late_payments INTEGER NOT NULL,
            defaults INTEGER NOT NULL,
            bankruptcies INTEGER NOT NULL,
            foreclosures INTEGER NOT NULL,
            outstanding_debt REAL NOT NULL
        )
    """)
    
    customers = [
        ("John Sample", 710, 1, 0, 0, 0, 5000.00),
        ("Jane Smith", 650, 3, 1, 0, 0, 15000.00),
        ("Alice Johnson", 810, 0, 0, 0, 0, 2000.00),
        ("Robert Brown", 580, 5, 2, 1, 1, 30000.00),
        ("Emily Davis", 700, 2, 0, 0, 0, 10000.00)
    ]
    
    cursor.executemany("""
        INSERT INTO customer (name, credit_score, late_payments, defaults, bankruptcies, foreclosures, outstanding_debt)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, customers)

    conn.commit()
    return conn

db_connection = setup_database()

def extract_customer_name(query_text: str) -> str:
    """
    Look up all customer names from the database and check if any of them appear in the query text.
    Returns the first matching customer name, or None if not found.
    """
    cursor = db_connection.cursor()
    cursor.execute("SELECT name FROM customer")
    names = [row[0] for row in cursor.fetchall()]
    
    for name in names:
        if re.search(r'\b' + re.escape(name) + r'\b', query_text, re.IGNORECASE):
            return name
    return None

class TextToSQLTool(BaseTool):
    name: str = "text_to_sql_tool"
    description: str = "Converts natural language queries about customer financial details into SQL and executes them on a SQLite DB."

    def _run(self, query: str) -> str:
        """
        Extracts the customer's name from the natural language query,
        constructs a SQL query to retrieve credit and financial details,
        executes the SQL query on the SQLite database, and returns the results.
        """

        query_text = query.strip()

        customer_name = extract_customer_name(query_text)
        
        if not customer_name:
            return "Could not extract customer name from the query."
        
        sql_query = """
            SELECT id, name, credit_score, late_payments, defaults, bankruptcies, foreclosures, outstanding_debt
            FROM customer WHERE name = ?;
        """
        try:
            cursor = db_connection.cursor()
            cursor.execute(sql_query, (customer_name,))
            result = cursor.fetchone()
            
            if result:
                id, name, credit_score, late_payments, defaults, bankruptcies, foreclosures, outstanding_debt = result
                return (
                    f"Customer ID: {id}\n"
                    f"Name: {name}\n"
                    f"Credit Score: {credit_score}\n"
                    f"Late Payments: {late_payments}\n"
                    f"Defaults: {defaults}\n"
                    f"Bankruptcies: {bankruptcies}\n"
                    f"Foreclosures: {foreclosures}\n"
                    f"Outstanding Debt: ${outstanding_debt:,.2f}"
                )
            else:
                return f"No record found for customer {customer_name}."
        except Exception as e:
            return f"Error executing SQL: {e}"

    def _arun(self, query: str) -> str:
        raise NotImplementedError("Asynchronous run not supported.")

#### Customer Information Verification and Task

- The Customer Information Verification Agent is responsible for retrieving and verifying a customer's full financial details using natural language queries converted into SQL. Leveraging TextToSQLTool, this agent ensures that structured financial data is accurately extracted for underwriting and risk assessment.

- The Information Verification Task fetches and validates customer financial records by executing SQL queries based on the provided customer name. It ensures that the retrieved information is structured according to a predefined JSON schema, making it easy for underwriters to analyze and assess loan eligibility.

In [6]:
information_verification_agent = Agent(
    role="Customer Information Verification Agent",
    goal="Retrieve and verify full customer financial details based on customer name, the name of the customer is extracted from the document and present in the ocr output under name key",
    backstory="An expert in financial risk assessment and underwriting, retrieving and verifying customer records for evaluation.",
    tools=[TextToSQLTool()],
    verbose=True,
    llm=LLM(model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"),
)

information_verification_task = Task(
    description="Fetch and verify all financial details for the customer mentioned in the query: {query}, use the tool results and create the expected final output",
    expected_output="""{
        "name": "string",
        "age": "integer",
        "SSN": "string",
        "phone_number": "string",
        "employment_information": {
            "employer": "string",
            "employer_address": "string",
            "position": "string",
            "years_employed": "integer"
        },
        "credit_information": {
            "credit_scores": "integer",
            "late_payments": "integer",
            "defaults": "integer",
            "bankruptcies": "integer",
            "foreclosures": "integer",
            "outstanding_debt": "float"
        },
        "property_information": {
            "current_residence": "string",
            "property_address": "string",
            "county": "string",
            "property_type": "string"
        },
        "loan_details": {
            "purchase_price": "float",
            "total_cost": "float",
            "loan_amount": "float"
        }
    }""",
    agent=information_verification_agent
)

#### KnowledgeBaseTool

The KnowledgeBaseTool is a CrewAI tool that enables retrieval of relevant information from an Amazon Bedrock Knowledge Base using the RetrieveAndGenerate API. It takes a user query as input, searches the knowledge base, and generates a response based on stored underwriting guidelines, financial policies, or other structured knowledge. This tool is essential for AI-driven loan underwriting, ensuring that underwriters have instant access to accurate, up-to-date regulatory and policy information. By integrating with AI agents, it enhances decision-making efficiency and reduces manual research efforts.

In [7]:
import boto3
from crewai.tools import BaseTool

class KnowledgeBaseTool(BaseTool):

    name: str = "knowledge_base_tool"
    description: str = "Tool to search the knowledge base using Amazon Bedrock's RetrieveAndGenerate API."

    def _run(self, search_question: str) -> str:
        try:
            bedrock_client = boto3.client("bedrock-agent-runtime")
            response = bedrock_client.retrieve_and_generate(
                input={'text': search_question},
                retrieveAndGenerateConfiguration={
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': 'WAUKT4P0KR',
                        'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0'
                    }
                }
            )
            return response['output']['text']
        except Exception as e:
            return f"An error occurred: {str(e)}"


#### Knowledge Base Agent and Task

- The Knowledge Base Agent retrieves contextual information from Amazon Bedrock's Knowledge Base to generate accurate and structured responses. It leverages LLM capabilities and retrieval-augmented generation (RAG) to ensure that underwriters and decision-makers have access to the latest financial and regulatory insights.

- The Knowledge Base Task processes user queries by searching the knowledge base and generating context-aware responses. It ensures that the retrieved information is relevant, structured, and accurate, assisting in loan underwriting and financial decision-making.

In [8]:
knowledge_base_agent = Agent(
    role="Knowledge Base Agent",
    goal="""Retrieve relevant contextual information from the knowledge base and generate accurate responses based on the search question""",
    backstory="An AI-powered agent designed to query Amazon Bedrock's Knowledge Base and provide well-structured responses leveraging LLM capabilities.",
    tools=[KnowledgeBaseTool()],
    verbose=True,
    llm=LLM(model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"),
)

knowledge_base_task = Task(
    description="Retrieve information from the knowledge base and generate a context-based response to the search question: '{search_question}'.",
    expected_output="A comprehensive and contextually relevant response based on the retrieved information.",
    agent=knowledge_base_agent
)

#### Underwriting Expert Agent and Task

The Underwriting Expert Agent evaluates applicant data against underwriting guidelines to provide risk-based recommendations. With expertise in financial risk assessment, it ensures well-informed and data-driven underwriting decisions.

The Underwriting Task analyzes applicant financials and compliance requirements to generate a clear underwriting decision with rationale. It ensures that loan approvals, rejections, or conditions are backed by logical explanations aligned with industry standards.

In [9]:
underwriting_expert = Agent(
    role="Underwriting Expert",
    goal="Provide underwriting recommendations based on applicant data and the underwriting guidelines.",
    backstory="You are experienced in risk assessment and underwriting decision-making.",
    verbose=True,
    llm=LLM(model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"),
)

underwriting_task = Task(
    description="Given applicant data  and the underwriting guidelines, provide an underwriting recommendation with rationale.",
    expected_output="An underwriting decision with supporting rationale.",
    agent=underwriting_expert
)

#### CrewAI-based underwriting system

- The Underwriting Crew is a fully automated AI-driven system designed to streamline the loan underwriting process by leveraging specialized agents working in a sequential pipeline. Each agent is responsible for a critical step in underwriting—starting with the OCR Agent, which extracts structured financial data from loan-related documents using Amazon Textract. The Information Verification Agent then queries financial databases, retrieving key details such as income, outstanding debts, and credit history using Text2SQL conversion. Next, the Knowledge Base Agent fetches relevant underwriting guidelines from Amazon Bedrock's knowledge base, ensuring compliance with risk assessment policies. Finally, the Underwriting Expert Agent synthesizes all extracted and verified information, applying industry-standard underwriting logic to provide a comprehensive loan recommendation.

- This CrewAI-based underwriting system ensures a seamless, accurate, and data-driven decision-making process, reducing the time and effort required for manual loan evaluation. By executing tasks sequentially, the system ensures that each underwriting step is backed by verified data, eliminating errors and inconsistencies. The use of LLMs and retrieval-augmented generation (RAG) allows for intelligent recommendations based on real-time financial information and underwriting policies. Whether processing new applications or reassessing loan risks, this AI-powered underwriting copilot enhances efficiency, ensures compliance, and provides clear, explainable loan decisions.



In [10]:
crew = Crew(
    agents=[ocr_agent,information_verification_agent, knowledge_base_agent, underwriting_expert],
    tasks=[ocr_task, information_verification_task, knowledge_base_task, underwriting_task],
    process=Process.sequential,
    verbose=True,
    llm=LLM(model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0")
)


inputs = {
    "query" : "Please extract all information of customer from the document",
    "file_path": "/home/sagemaker-user/underwriting_copilot/docs/",
    "search_question": "provide detailed underwriting guidelines"
}


result = crew.kickoff(inputs=inputs)
print(result)

# Agent: OCR Agent
## Task: Extract and structure data from the document at path: /home/sagemaker-user/underwriting_copilot/docs/. Do not hallucinate and do not include information thats not in the documents


# Agent: OCR Agent
## Thought: I'll systematically process the document using the Textract OCR tool to extract the required information.
## Using tool: textract_ocr_tool
## Tool Input: 
"{\"file_path\": \"/home/sagemaker-user/underwriting_copilot/docs/underwriting_document.jpg\"}"
## Tool Output: 
Error during OCR extraction: [Errno 2] No such file or directory: '/home/sagemaker-user/underwriting_copilot/docs/underwriting_document.jpg'


# Agent: OCR Agent
## Thought: Thought: I understand that I need to carefully review the task requirements and be prepared to extract information systematically.
## Using tool: textract_ocr_tool
## Tool Input: 
"{\"file_path\": \"/home/sagemaker-user/underwriting_copilot/docs/\"}"
## Tool Output: 
--- Text from loan-application-pg3.png ---
YOUR C

### Agent Eval with RAGAS

Agentic or tool use workflows can be evaluated in multiple dimensions.

#### Tool call Accuracy

ToolCallAccuracy is a metric that can be used to evaluate the performance of the LLM in identifying and calling the required tools to complete a given task. This metric needs user_input and reference_tool_calls to evaluate the performance of the LLM in identifying and calling the required tools to complete a given task. The metric is computed by comparing the reference_tool_calls with the Tool calls made by the AI. The values range between 0 and 1, with higher values indicating better performance.

In [11]:
from langchain_aws import ChatBedrock as LangChainBedrock

In [12]:
from ragas.dataset_schema import MultiTurnSample
from ragas.messages import HumanMessage, AIMessage, ToolCall
import json

# Define the evaluation samples
evaluation_samples = [
    MultiTurnSample(
        user_input=[
            HumanMessage(content="Extract and aggregate text from all documents in the specified folder using Amazon Textract"),
            AIMessage(content="Extracting information for customer", tool_calls=[
                ToolCall(name="TextractOCRTool", args={"file_path": "/home/sagemaker-user/underwriting_copilot/docs/"})
            ])
        ],
        reference_tool_calls=[
            ToolCall(name="TextractOCRTool", args={"file_path": "/home/sagemaker-user/underwriting_copilot/docs/"})
        ],
    ),
    # Add more samples as needed
]


In [13]:
from ragas import EvaluationDataset

# Create the evaluation dataset
evaluation_dataset = EvaluationDataset(samples=evaluation_samples)

In [14]:
from ragas.metrics import ToolCallAccuracy

# Initialize metrics
tool_call_accuracy_metric = ToolCallAccuracy()

In [15]:
from ragas import evaluate
from langchain_aws import ChatBedrock as LangChainBedrock

# Evaluate the agent
evaluation_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[tool_call_accuracy_metric],
    llm=LangChainBedrock(model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0")
)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
evaluation_result

{'tool_call_accuracy': 1.0000}